# Assigment 9 | Applied Machine Learning | Paras Ahuja

### Please refer to the Reinforcement Learning Jupyter notebook in course materials

### 1. Describe the environment in the Nim learning model.

Kai Arulkumaran, Marc Peter Deisenroth, Miles Brundage, and Anil Anthony Bharat state in their paper entitled "Deep Reinforcement Learning: A brief survey ": in the  RL  setup,  an  autonomous  agent,  controlled  by  a  machine-learning algorithm, observes a statest from its environment at time step $t$ The agent interacts with the environment by taking an action $a_t$ in state $s_t$. When the agent takes an action, the environment and the agent transition to a new state, $s_{t+1}$ based on the current state and the chosen action. 

**The environment in the game of Nim is the board itself.** In the Nim learning model we have the Nim game of strategy between two players in which players remove items from three piles at each turn. Every turn the player must remove at least one item from a pile. There are two versions of the game goal: 
1. the player who clears the last pile wins
2. the player who clears the last pile (by removing >1 items) loses.

### 2. Describe the agent in the Nim learning model.

The goal of the agent is to learn a policy(control strategy) $\pi$ that maximizes the expected return (cumulative, discounted reward). Our agent is Q-Learning algorithm or the Q-Learning agent. At the start of the game or training phase the entries in the agent’s Q-table are zero. The agent is  programmed to read the board and choose an action according to the algorithm. Generally speaking, if the procedure produces an illegal move then the agent should make a random legal move instead. Essentially, the agent is using the qtable for expected payoffs to make guesses about the stack. Our agent is programmed to gravitate towards the highest expected reward. 

### 3. Describe the reward and penalty in the Nim learning model.

The reward is given to the agent after each pair of moves: the agent moves, then the opponent moves, in this way the agent learns the ultimate consequences of its actions and the agent's qtable is updated accordingly. If on the other hand the opponent made the first move of the game, the agent’s Q-table is first updated after the opponent made its second move. Generally speaking in many Q-learning implementations, our goal is to learn a reward value for every (state, action) pair. An action that loses the game will have a reward of -1, an action that results in the other player losing the game will have a reward of 1, and an action that results in the game continuing has an immediate reward of 0, but will also have some future reward.

Reference:https://cs50.harvard.edu/extension/ai/2020/spring/projects/4/nim/


### 4. How many possible states there could be in the Nim game with a maximum of 10 items per pile and 3 piles total?

A state is any permutation of the board. The number of individual states possible in a game of Nim can be easily calculated. For a board containing three heaps of $x_1, x_2, \text{and} x_3$ counters the total number of states is given by $(x_1+1)(x_2+1)(x_3 +1)$. Then, we have $(10+1)(10+1)(10+1) = 11^3 = 1331.$ Therefore, the game has 1331 states - thats a lot of states.

### 5. How many possible actions there could be in the Nim game with 10 items per pile and 3 piles total?

Agents interact with the environment by removing counters from the board. From a heap of x counters an agent can take anything from 1 up to x counters. The agent must however only take counters from one heap in each move. The total number of actions available to the agent is therefore equal to $x_1 + x_2 + x_3$. Then, we have $10 + 10 + 10 = 30$. Therefore, we have 30 possible actions in the game.

### 6. Find a way to improve the provided Nim game learning model. Do you think one can beat the Guru player? (Hint: How about penalizing the losses? Hint: It is indeed possible to find a better solution, which improves the way Q-learning updates its Q-table).

It is entirely possible for any of the 3 agent to learn the optimal strategy of Nim. We notice that the strategy below improves the algorithm by a bit as compared to default. This means that there is definitely a method will can make the algorithm learn the policy that is highly comparable to guru learner. Furthermore, we have to note the point that if Guru goes first, it will win vast majority of the time, at least with this iteration of the algorithm. However, if Qlearner goes first, it wins more than it would otherwise. We notice that in one iteration of the game, Qlearner manages to win 105 times against Guru learner, however Qlearner went first. Still, there is more room for improvement. 

In [1]:
import numpy as np
from random import randint, choice

# Who won
won = "Guru"

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game():
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(st):
    xored = st[0] ^ st[1] ^ st[2]
    if xored == 0:
        return nim_random(st)
    #
    for pile in range(3):
        s = st[pile] ^ xored
        if s <= st[pile]:
            return st[pile]-s, pile

# Random Nim player
def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

In [2]:
def nim_qlearner(_st):
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    #
    return move, pile  # action


In [3]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(a, b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[a] if side == 'A' else Engines[b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side
        #
        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n, a, b):
    from collections import defaultdict
    wins = defaultdict(int)
    for i in range(_n):
        wins[game(a, b)] += 1
    # info
    print(f"{_n} games, {a:>8s} {wins['A']}   {b:>8s} {wins['B']}")
    #
    won = a if wins['A'] > wins['B'] else b
    return wins['A'], wins['B']

In [4]:
# Play games
play_games(1000, 'Random', 'Random')
play_games(1000, 'Guru', 'Random')
play_games(1000, 'Random', 'Guru')
play_games(1000, 'Guru', 'Guru') ;

1000 games,   Random 492     Random 508
1000 games,     Guru 999     Random 1
1000 games,   Random 6       Guru 994
1000 games,     Guru 950       Guru 50


In [5]:
qtable, Alpha, Gamma, Reward = None, 1, 0.8, 100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            nim_sum = st2[0] ^ st2[1] ^ st2[2]
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game  
            elif nim_sum != 0:
                qtable_update(-Reward, st1, move, pile, np.min(qtable[st2[0], st2[1], st2[2]]))
            else:
                qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

In [6]:
%%time
nim_qlearn(100)

CPU times: user 6.75 ms, sys: 2.19 ms, total: 8.94 ms
Wall time: 6.73 ms


In [7]:
# Play games
play_games(1000, 'Qlearner', 'Random')
play_games(1000, 'Random', 'Qlearner') ;

1000 games, Qlearner 731     Random 269
1000 games,   Random 263   Qlearner 737


In [8]:
%%time
# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
wins = []
for n in n_train:
    nim_qlearn(n)
    a, b = play_games(1000, 'Guru', 'Qlearner')
    wins += [a/(a+b)]

1000 games,     Guru 998   Qlearner 2
1000 games,     Guru 999   Qlearner 1
1000 games,     Guru 998   Qlearner 2
1000 games,     Guru 995   Qlearner 5
1000 games,     Guru 991   Qlearner 9
1000 games,     Guru 994   Qlearner 6
1000 games,     Guru 989   Qlearner 11
CPU times: user 7.5 s, sys: 838 ms, total: 8.34 s
Wall time: 7.62 s


In [9]:
# Check the ratio of wins wrt to size of the reinforcement model training
print(wins)

[0.998, 0.999, 0.998, 0.995, 0.991, 0.994, 0.989]


In [10]:
# Function to print the entire set of states
def qtable_log(_fn):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)
        #
        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r
            #
            print(s, file=fout)
#
qtable_log('qtable_debug.txt')